In [ ]:

# Here we gonna implement a feedforward network (multilayer nn) which can do classification
# based on MNIST dataset. The steps are -

# load MNIST dataset
# Dataloader, Transformation
# Multilayer neural network, Activation Function
# Loss and optimiser
# Training Loop
# Model evaluation
# GPU support

# memorise everything!! 

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.utils.data.dataloader

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784 # 28x28
hidden_size = 500 # change and see what happens
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../data/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../data/', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

examples = iter(train_loader)
example_data, example_targets = next(examples) # 100, 1, 28, 28; 100 is  batch size, 1 is 1 colour channel ie grap, 28 * 28 image resolution 

print(f"samples size: {example_data.shape}, target size: {example_targets.shape}")


# for i in range(9):
#     plt.subplot(3,3,i+1)
#     plt.imshow(example_data[i][0], cmap='gray')
# plt.show()

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out
        
# model, loss, optimiser

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()

optimser = torch.optim.Adam(model.parameters(), lr=learning_rate)

# training loop
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    n_sample = 0
    for i, (images, labels) in enumerate(train_loader):
        # 100, 1, 28, 28 and input size is (100, 784) so reshape is required
        images = images.reshape(-1, 28*28).to(device) # -1 here is 28
        labels = labels.to(device)
        
        # forward pass
        pred_out = model(images).to(device)
        loss = criterion(pred_out, labels)
        
        #backwards
        optimser.zero_grad()
        loss.backward()
        optimser.step()
        
        n_sample += images.shape[0]
        
        if ((i+1) % 100) == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')    
    
    
# Test model 

with torch.no_grad():
    n_correct_pred = 0
    n_sample = 0
    
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # value, index
        _, predictions = torch.max(outputs, 1)
        n_sample += labels.shape[0]
        n_correct = (predictions == labels).sum().item()
        
    acc = 100.0 * n_correct / n_sample
    print(f"accuracy = {acc}")

/Users/vishal/opt/anaconda3/envs/fincode/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/vishal/opt/anaconda3/envs/fincode/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
